## IMPORTING REQUIRED LIBRARIES

In [1]:
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import re
import time as t
from datetime import datetime

# CLEANED.JSONL

In [2]:
jsonl_path = 'C:/Users/sidew/Desktop/gearbox/cleaned.jsonl'

df = pd.read_json(jsonl_path, lines = True)
# df2 = pd.read_json('C:/Users/sidew/Desktop/gearbox/cleaned2.jsonl', lines = True)

## NORMALIZING NESTED JSON DATA STORED IN THE FIRST COLUMN - 'CONTEXT'

In [3]:
aass = json_normalize(df['context'])
aass

,timestamp,session_guid,hardware,map
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P
1,1583797550,0B7152ED08D7C44BF565BA0A0A05DE46,ps4,OrbitalPlatform_P
2,1583797550,0C5886A808D7C466611743330B3CF5E3,ps4,Watership_P
3,1583797550,0BD437A408D7C457E71407DA0A687388,ps4,OrbitalPlatform_P
4,1583797550,DEB3B53A4677DB2AC377498E9283E10E,xboxone,MarshFields_P
...,...,...,...,...
118199,1583816535,22C533B2485A92B39CBCF7973C7984EC,xboxone,Sanctuary3_P
118200,1583816536,C2FF691548C47653B0D4E6BE93A89697,xboxone,Wetlands_P
118201,1583816536,D11852E243CFD562CC0FF7A4B67BC4A0,xboxone,WetlandsBoss_P
118202,1583816536,0C64731008D7C5074F29482A08FF25A2,ps4,Sanctuary3_P


## NORMALIZING NESTED JSON DATA STORED IN THE SECOND COLUMN - 'WEAPONS'

In [4]:
sass = pd.concat([pd.DataFrame(json_normalize(x)) for x in df['weapons']])
sass

,class,fired,criticals,hits,damage,aoe_damage,crit_damage,reloads,trigger_pulls,type
0,BPChar_Operative_C,66,1,None,891130.00,0.00,35231.0,0,33,WT_PS_MAL
1,BPChar_Operative_C,0,0,None,4107.09,0.00,0.0,0,0,DamageSource
2,BPChar_Operative_C,0,0,None,50678.40,0.00,0.0,0,0,DamageSource_Skill_Operative_Drone_C
3,BPChar_Operative_C,0,0,None,0.00,5995.50,0.0,0,0,DamageSource_Grenade_C
4,BPChar_Operative_C,0,0,None,0.00,6187.67,0.0,0,0,DamageSource_GrenadeDoT_C
...,...,...,...,...,...,...,...,...,...,...
0,BPChar_Siren_C,0,0,None,5053.65,0.00,0.0,0,0,DamageSource_StatusEffect_C
1,BPChar_Siren_C,4,0,None,76638.10,17154.50,0.0,2,2,WT_SG_MAL
2,BPChar_Siren_C,0,0,None,301807.00,1181890.00,0.0,0,0,DamageSource_Bullet_Shotgun_C
3,BPChar_Siren_C,0,0,None,1290490.00,347745.00,0.0,0,0,DamageSource_Bullet_C


## UNPACKING MULTIPLE ROWS OF DATA WHILE PRESERVING THE INDEX NUMBER

In [5]:
index_list = sass.index.tolist()
idx_list = sass.index.tolist()
count = 0
pos = 0
for i in range(1,len(index_list)):
    if index_list[i] == 0:
        for j in range(pos,i):
            index_list[j] = count
        count+=1
        index_list[i] = count
        pos = i
for k in range(pos, len(index_list)):
    index_list[k] = count

In [6]:
sass.index = index_list
sass

,class,fired,criticals,hits,damage,aoe_damage,crit_damage,reloads,trigger_pulls,type
0,BPChar_Operative_C,66,1,None,891130.00,0.00,35231.0,0,33,WT_PS_MAL
0,BPChar_Operative_C,0,0,None,4107.09,0.00,0.0,0,0,DamageSource
0,BPChar_Operative_C,0,0,None,50678.40,0.00,0.0,0,0,DamageSource_Skill_Operative_Drone_C
0,BPChar_Operative_C,0,0,None,0.00,5995.50,0.0,0,0,DamageSource_Grenade_C
0,BPChar_Operative_C,0,0,None,0.00,6187.67,0.0,0,0,DamageSource_GrenadeDoT_C
...,...,...,...,...,...,...,...,...,...,...
118203,BPChar_Siren_C,0,0,None,5053.65,0.00,0.0,0,0,DamageSource_StatusEffect_C
118203,BPChar_Siren_C,4,0,None,76638.10,17154.50,0.0,2,2,WT_SG_MAL
118203,BPChar_Siren_C,0,0,None,301807.00,1181890.00,0.0,0,0,DamageSource_Bullet_Shotgun_C
118203,BPChar_Siren_C,0,0,None,1290490.00,347745.00,0.0,0,0,DamageSource_Bullet_C


## MERGING THE COLUMNS UNPACKED FROM THE NESTED JSON COLUMNS IN THE ORIGINAL FILE USING THE INDEX NUMBER AS THE JOINING REFERENCE

In [7]:
dass = pd.merge(aass, sass, left_index=True, right_index=True)
dass

,timestamp,session_guid,hardware,map,class,fired,criticals,hits,damage,aoe_damage,crit_damage,reloads,trigger_pulls,type
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P,BPChar_Operative_C,66,1,None,891130.00,0.00,35231.0,0,33,WT_PS_MAL
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P,BPChar_Operative_C,0,0,None,4107.09,0.00,0.0,0,0,DamageSource
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P,BPChar_Operative_C,0,0,None,50678.40,0.00,0.0,0,0,DamageSource_Skill_Operative_Drone_C
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P,BPChar_Operative_C,0,0,None,0.00,5995.50,0.0,0,0,DamageSource_Grenade_C
0,1583797550,180237AD47320869A9F18CAE3B149753,pc,City_P,BPChar_Operative_C,0,0,None,0.00,6187.67,0.0,0,0,DamageSource_GrenadeDoT_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118203,1583816536,7A4886D44016791E9795C7AE63E97E24,xboxone,Beach_P,BPChar_Siren_C,0,0,None,5053.65,0.00,0.0,0,0,DamageSource_StatusEffect_C
118203,1583816536,7A4886D44016791E9795C7AE63E97E24,xboxone,Beach_P,BPChar_Siren_C,4,0,None,76638.10,17154.50,0.0,2,2,WT_SG_MAL
118203,1583816536,7A4886D44016791E9795C7AE63E97E24,xboxone,Beach_P,BPChar_Siren_C,0,0,None,301807.00,1181890.00,0.0,0,0,DamageSource_Bullet_Shotgun_C
118203,1583816536,7A4886D44016791E9795C7AE63E97E24,xboxone,Beach_P,BPChar_Siren_C,0,0,None,1290490.00,347745.00,0.0,0,0,DamageSource_Bullet_C


In [8]:
dass.isna().sum()

timestamp              0
session_guid           0
hardware               0
map                    0
class                  0
fired                  0
criticals              0
hits             1070501
damage                 0
aoe_damage             1
crit_damage            0
reloads                0
trigger_pulls          0
type                   0
dtype: int64

## DATA TYPE CONVERSION - ENSURING THE COLUMNS ARE STORED IN THE APPROPRIATE FORMAT

In [94]:
dass['DATETIME'] = [datetime.fromtimestamp(i) for i in dass['timestamp']]

## REARRANGING ORDER OF COLUMNS IN A MORE MEANINGFUL MANNER AND DROPPING COLUMNS WITH NO VARIATION

In [133]:
finale_js = dass.iloc[:,[14,1,2,3,4,5,6,7,8,9,10,11,12,13]]
del finale_js['hits']
finale_js = finale_js.dropna()

## COMBINING FACTORS IN A COLUMN TO REDUCE THE TOTAL NUMBER OF FACTORS IN EACH COLUMN - DONE IN A MEANINGFUL MANNER W.R.T THE DOMAIN

In [135]:
clsid = {'BPChar_Beastmaster_C':'beastmaster', 'BPChar_Gunner_C':'gunner','BPChar_IronBear_C':'gunner','BPChar_IronBear_DakkaBear_C':'gunner',
          'BPChar_Siren_C':'siren', 'BPChar_Operative_C':'operative',
          'Vehicle_Outrunner_HoverWheels_C':'outrunner','Vehicle_Outrunner_C':'outrunner','Vehicle_Outrunner_ZipWheels_C':'outrunner','Vehicle_Outrunner_TwitchyWheels_C':'outrunner','Vehicle_GunnerSeat_Outrunner_C':'outrunner',
          'Vehicle_Technical_BigWheels_C':'technical','Vehicle_Technical_C':'technical','Vehicle_Technical_HoverWheels_C':'technical','Vehicle_Technical_BarbedWheels_C':'technical','Vehicle_GunnerSeat_Technical_C':'technical',
          'Vehicle_Revolver_C':'revolver','Vehicle_Revolver_DualWheel_C':'revolver','Vehicle_Revolver_HoverWheels_C':'revolver','Vehicle_Revolver_WideWheel_C':'revolver',
          'IO_GroundTurret_C':'groundturret','BPSeatPawn_GunnersNest_C':'gunnersnest','NONE':np.nan}

In [136]:
finale_js = finale_js.replace({'class':clsid})

In [137]:
finale_js['class'].unique()

array(['operative', 'gunner', 'beastmaster', 'revolver', 'siren',
       'technical', 'outrunner', 'IO_GroundTurret', 'gunnersnest', nan],
      dtype=object)

In [139]:
maps = {'ProvingGrounds_Trial4_P':'provinggrounds','ProvingGrounds_Trial5_P':'provinggrounds','ProvingGrounds_Trial8_P':'provinggrounds',
       'ProvingGrounds_Trial6_P':'provinggrounds','ProvingGrounds_Trial1_P':'provinggrounds','ProvingGrounds_Trial7_P':'provinggrounds',
       'COVSlaughter_P':'circleofslaughter','CreatureSlaughter_P':'circleofslaughter','TechSlaughter_P':'circleofslaughter','MotorcadeInterior_P':'motorcade',
       'Motorcade_P':'motorcade','MotorcadeFestival_P':'motorcade','City_P':'city','CityVault_P':'city','CityBoss_P':'city','Desert_P':'desert',
       'Desertvault_P':'desert','DesertBoss_P':'desert','WetlandsBoss_P':'wetlands','Wetlands_P':'wetlands','WetlandsVault_P':'wetlands'}

In [141]:
finale_js = finale_js.replace({'map':maps})

In [142]:
finale_js['map'].unique()

array(['city', 'OrbitalPlatform_P', 'Watership_P', 'MarshFields_P',
       'Sanctuary3_P', 'Raid_P', 'Prison_P', 'wetlands', 'Monastery_P',
       'Outskirts_P', 'AtlasHQ_P', 'Sacrifice_P', 'motorcade', 'Beach_P',
       'Prologue_P', 'Desolate_P', 'Loader', 'Mine_P', 'Core_P', 'desert',
       'Trashtown_P', 'Towers_P', 'CasinoIntro_P', 'circleofslaughter',
       'Mansion_P', 'Strip_P', 'provinggrounds', 'Crypt_P', 'TowerLair_P',
       'Impound_P', 'Recruitment_P', 'FinalBoss_P', 'Convoy_P',
       'MenuMap_P', 'Village_P', 'Camp_P'], dtype=object)

In [144]:
typed = {'WT_PS_MAL':'pistol','WT_SG_TOR':'shotgun','WT_AR_DAL':'assaultrifle','WT_SM_HYP':'smg','WT_PS_JAK':'pistol','WT_SM_MAL':'smg',
        'WT_PS_TOR':'pistol','WT_AR_JAK':'assaultrifle','WT_SG_HYP':'shotgun','WT_AR_VLA':'assaultrifle','WT_PS_VLA':'pistol','WT_SM_DAL':'smg',
        'WT_PS_COV':'pistol','WT_AR_TOR':'assaultrifle','WT_EridianFabricator':'eridianfabricator','WT_PS_MAL_SuckerPunch':'pistol','WT_SG_JAK':'shotgun',
        'WT_SG_MAL':'shotgun','WT_AR_ATL':'assaultrifle','WT_PS_TED':'pistol','WT_PS_DAL':'pistol','WT_PS_ATL':'pistol','WT_SM_HYP_Handsome':'smg',
        'WT_SM_TED':'smg','WT_AR_COV':'assaultrifle','WT_SG_TED':'shotgun','DamageSource_Skill_Operative_Drone_C':'operative','DamageSource_Grenade_C':'grenade',
        'DamageSource_GrenadeDoT_C':'grenadedot','DamageSource_Skill_StaticField_C':'skillstaticfield','DamageSource_StatusEffect_C':'statuseffect',
        'DamageSource_Shield_C':'shield','BPChar_Agonizer_9k_C':'Agonizer','BPChar_AnointedJoeClone_C':'AnointedJoeClone','BPChar_ApeAcid_C':'Ape',
        'BPChar_ApeBadass_C':'Ape','BPChar_ApeBasic_C':'Ape','BPChar_ApeChump_C':'Ape','BPChar_ApeCryo_C':'Ape','BPChar_ApeFire_C':'Ape',
        'BPChar_ApeKGuard_C':'Ape','BPChar_ApeLoot_C':'Ape','BPChar_ApeRad_C':'Ape','BPChar_ApeShock_C':'Ape','BPChar_ApeSquire_C':'Ape',
        'BPChar_AtlasSoldier_BountyGang_C':'AtlasSoldier','BPChar_Beastmaster_C':'beastmaster','BPChar_BehemothGunner_C':'behemoth',
        'BPChar_BehemothRaid_C':'behemoth','BPChar_BehemothRocketeer_C':'behemoth','BPChar_BrainBeam_C':'brainbeam','BPChar_CasinoBot_WMS_C':'casinobot',
        'BPChar_ClaptrapSwarm_C':'claptrapswarm','BPChar_ConstructorBasic_C':'constructorbasic','BPChar_DigiClone_Badass_C':'digiclone',
        'BPChar_DigiClone_Normal_C':'digiclone','BPChar_DigiClone_SuperBadass_C':'digiclone','BPChar_DigiClone_UltimateSuperBadass_C':'digiclone',
        'BPChar_DropShipTurret_C':'dropshipturret','BPChar_Enforcer_BountyPrologue_C':'enforcer','BPChar_EnforcerBadass_C':'enforcer','BPChar_EnforcerBadass_Looter_C':'enforcer',
        'BPChar_EnforcerBasic_C':'enforcer','BPChar_EnforcerBruiser_Looter_C':'enforcer','BPChar_EnforcerGun_C':'enforcer','BPChar_EnforcerKillaVolt_C':'enforcer',
        'BPChar_EnforcerMelee_C':'enforcer','BPChar_EnforcerMeleeWardenGuard_C':'enforcer','BPChar_FinalBoss_C':'finalboss','BPChar_Frontrunner_Badass_C':'frontrunner',
        'BPChar_FrontrunnerBasic_C':'frontrunner','BPChar_FrontrunnerBasic_Raid1_C':'frontrunner','BPChar_FrontrunnerJammer_C':'frontrunner',
        'BPChar_FrontrunnerStriker_C':'frontrunner','BPChar_Goliath_Badass_C':'goliath','BPChar_Goliath_Basic_C':'goliath','BPChar_Goliath_CageArena_C':'goliath',
        'BPChar_Goliath_Corrosive_C':'goliath','BPChar_Goliath_Midget_C':'goliath','BPChar_Goliath_Rare01_C':'goliath','BPChar_Goliath_Rare03_C':'goliath',
        'BPChar_Goliath_SlaughterBoss_C':'goliath','BPChar_GoliathBasic_Looter_C':'goliath','BPChar_GoliathMidget_Looter_C':'goliath','BPChar_GoonAnointed_C':'goon',
        'BPChar_GoonBadass_C':'goon','BPChar_GoonBasic_C':'goon','BPChar_GoonVortex_C':'goon','BPChar_Guardian_TrialBoss_C':'guardian',
        'BPChar_GuardianGemGoblin_C':'guardian','BPChar_GuardianHerald_C':'guardian','BPChar_GuardianSera_C':'guardian','BPChar_GuardianSpectre_C':'guardian',
        'BPChar_GuardianWraith_C':'guardian','BPChar_GuardianWraithBossFodder_C':'guardian','BPChar_Gunner_C':'gunner','BPChar_Heavy_Acidrain_C':'heavy',
        'BPChar_Heavy_AcidrainDark_C':'heavy','BPChar_Heavy_AcidrainDarkRaid1_C':'heavy','BPChar_Heavy_Badass_C':'heavy','BPChar_Heavy_BadassDark_C':'heavy',
        'BPChar_Heavy_BadassDarkRaid1_C':'heavy','BPChar_Heavy_Basic_C':'heavy',' BPChar_Heavy_BasicDark_C':'heavy','BPChar_Heavy_BasicDarkRaid1_C':'heavy',
        'BPChar_Heavy_Bounty01_C':'heavy','BPChar_Heavy_Icebreaker_C':'heavy','BPChar_Heavy_IcebreakerDark_C':'heavy','BPChar_Heavy_IcebreakerDarkRaid1_C':'heavy',
        'BPChar_Heavy_Powerhouse_C':'heavy','BPChar_Heavy_PowerhouseDark_C':'heavy','BPChar_Heavy_PowerhouseDarkRaid1_C':'heavy','BPChar_Heavy_Traunt_C':'heavy',
        'BPChar_HeavyDarkTraunt_C':'heavy','BPChar_HeavyGunner_C':'heavy','BPChar_HeavyGunnerDark_C':'heavy','BPChar_IronBear_C':'gunner','BPChar_IronBear_DakkaBear_C':'gunner',
        'BPChar_JackBot_C':'jackbot','BPChar_LoaderBadass_C':'loader','BPChar_LoaderBasic_C':'loader','BPChar_LoaderBasicJUNK_C':'loader',
        'BPChar_LoaderElementalCryo_C':'loader','BPChar_LoaderEXP_C':'loader','BPChar_LoaderEXP_JUNK_C':'loader','BPChar_LoaderRIOT_C':'loader',
        'BPChar_LoaderRPG_C':'loader','BPChar_LoaderSGT_C':'loader','BPChar_LoaderSGT_JUNK_C':'loader','BPChar_LoaderSVY_C':'loader','BPChar_Lorelei_C':'lorelei',
        'BPChar_MechBasicDark_C':'mech','BPChar_MechEvilAI_C':'mech','BPChar_MechGenMini_C':'mech','BPChar_MechMGDark_C':'mech','BPChar_MechRaidBossB_C':'mech',
        'BPChar_Nekrobug_Badass_C':'nekrobug','BPChar_Nekrobug_BadVibes_C':'nekrobug','BPChar_Nekrobug_Bomber_C':'nekrobug','BPChar_Nekrobug_Broodling_C':'nekrobug',
        'BPChar_Nekrobug_Flyer_C':'nekrobug','BPChar_Nekrobug_Ground_C':'nekrobug','BPChar_Nekrobug_Hopper_C':'nekrobug','BPChar_Nekrobug_Hunt01_C':'nekrobug',
        'BPChar_NogBadass_C':'nog','BPChar_NogBadassDark_C':'nog','BPChar_NogBasic_C':'nog','BPChar_NogBasicDark_C':'nog','BPChar_NogBeans_C':'nog',
        'BPChar_NogChipHolder_C':'nog','BPChar_NogNogromancer_C':'nog','BPChar_Operative_C':'operative','BPChar_Oversphere_C':'oversphere',
        'BPChar_OversphereDark_C':'oversphere','BPChar_OversphereDeathsphere_C':'oversphere','BPChar_OversphereDefender_C':'oversphere',' BPChar_OversphereDefenderDarkRaid1_C':'oversphere',
        'BPChar_OversphereStinger_C':'oversphere','BPChar_PetMonkey_C':'petmonkey','BPChar_PetMonkey_Evo1_Beefcake_C':'petmonkey','BPChar_PetMonkey_Evo2_Gunslinger_C':'petmonkey',
        'BPChar_PetSkag_C':'petskag','BPChar_PetSkag_Evo1_Horned_C':'petskag','BPChar_PetSkag_Evo2_Eridian_C':'petskag','BPChar_PetSpiderant_C':'petspider',
        'BPChar_PetSpiderant_Evo1_Fire_C':'petspider','BPChar_PetSpiderant_Evo2_King_C':'petspider','BPChar_PsychoAnointed_C':'psycho',
        'BPChar_PsychoBadass_C':'psycho','BPChar_PsychoBasic_C':'psycho','BPChar_PsychoBasic_Looter_C':'psycho','BPChar_PsychoFirebrand_C':'psycho',
        'BPChar_PsychoFirebrand_Looter_C':'psycho','BPChar_PsychoInfectedOnes_C':'psycho','BPChar_PsychoSlugger_C':'psycho','BPChar_PsychoSlugger_Looter_C':'psycho',
        'BPChar_PsychoSuicide_C':'psycho','BPChar_PsychoSuicide_Looter_C':'psycho','BPChar_PsychoWardenGuard_C':'psycho','BPChar_PunkArmored_LooterVIP_C':'punk',
        'BPChar_PunkAssaulter_C':'punk','BPChar_PunkAssaulter_Looter_C':'punk','BPChar_PunkBadass_C':'punk','BPChar_PunkBadassArmored_C':'punk',
        'BPChar_PunkBasic_C':'punk','BPChar_PunkBasic_Looter_C':'punk','BPChar_PunkPrologue_C':'punk','BPChar_PunkShotgunner_C':'punk','BPChar_PunkShotgunner_Looter_C':'punk',
        'BPChar_PunkSniper_C':'punk','BPChar_PunkSniper_Looter_C':'punk','BPChar_Rakk_Corrosive_C':'rakk','BPChar_Rakk_Hunt01_C':'rakk',
        'BPChar_RakkBasic_C':'rakk','BPChar_RakkBossFodder_C':'rakk','BPChar_RakkChromatic_C':'rakk','BPChar_RakkFlightless_Radioactive_C':'rakk',
        'BPChar_RakkNeedler_C':'rakk','BPChar_RakkQueen_C':'rakk','BPChar_Rampager_C':'rampager','BPChar_Ratch_Hunt01_C':'ratch',
        'BPChar_RatchBadassRaid1_C':'ratch','BPChar_RatchBadRat_C':'ratch','BPChar_RatchBasic_C':'ratch','BPChar_RatchBasicRaid1_C':'ratch',
        'BPChar_RatchBirther_C':'ratch','BPChar_RatchEgg_Basic_C':'ratch','BPChar_RatchEgg_Pup_C':'ratch','BPChar_RatchGnat_C':'ratch',
        'BPChar_RatchLarva_C':'ratch','BPChar_RatchPup_C':'ratch','BPChar_RatchPupRaid1_C':'ratch','BPChar_RatchSpaceSlug_C':'ratch','BPChar_RatchSpitter_C':'ratch',
        'BPChar_RatchSwarm_C':'ratch','BPChar_RegainingFeet_GoldenBullion_C':'regainfeetgoldbullion','BPChar_Saurian_Grog_C':'saurian','BPChar_Saurian_Grog_Fire_C':'saurian',
        'BPChar_Saurian_Grog_Poison_C':'saurian','BPChar_Saurian_Hamtaurus_C':'saurian','BPChar_Saurian_Predator_C':'saurian','BPChar_Saurian_Predator_X_C':'saurian',
        'BPChar_Saurian_Pygmimus_C':'saurian','BPChar_Saurian_Rare01_C':'saurian','BPChar_Saurian_Tyrant_C':'saurian','BPChar_SaurianLaser_C':'saurian',
        'BPChar_SaurianQueen_C':'saurian','BPChar_SaurianShield_C':'saurian','BPChar_ServiceBot_Basic_C':'servicebot','BPChar_ServiceBot_Bouncer_C':'servicebot',
        'BPChar_ServiceBot_EMS_C':'servicebot','BPChar_ServiceBot_FIRE_C':'servicebot','BPChar_ServiceBot_JanitorBot_C':'servicebot','BPChar_ServiceBot_SWAT_C':'servicebot',
        'BPChar_ServiceBot_WMS_C':'servicebot','BPChar_Siren_C':'siren','BPChar_Skag_Rare01_C':'skag','BPChar_SkagAdult_C':'skag','BPChar_SkagAlpha_C':'skag','BPChar_SkagBadass_C':'skag',
        'BPChar_SkagBarfer_C':'skag','BPChar_SkagPup_C':'skag','BPChar_SkagSpitter_C':'skag','BPChar_SpiderantBasic_C':'spiderant','BPChar_SpiderantBishop_C':'spiderant',
        'BPChar_SpiderantQueen_C':'spiderant','BPChar_SpiderantRook_C':'spiderant','BPChar_SpiderantSpiderling_C':'spiderant','BPChar_SpiderantTarantella_C':'spiderant',
        'BPChar_SurveyorBasic_C':'surveyor',' BPChar_TedioreTurret_PS_C':'tedturret','BPChar_TedioreTurret_PS_Sabre_C':'tedturret','BPChar_TedioreTurret_SG_C':'tedturret',
        'BPChar_TedioreTurret_SMG_C':'tedturret','BPChar_TedioreTurret_SpiderMind_C':'tedturret','BPChar_TherapyBot_C':'therapybot','BPChar_Tink_SentryRocketPodBasic_Pile_C':'tink',
        'BPChar_Tink_TurretBasic_C':'tink','BPChar_Tink_TurretBasicPile_C':'tink','BPChar_TinkAnointed_C':'tink','BPChar_TinkArchimedes_C':'tink','BPChar_TinkBadass_C':'tink',
        'BPChar_TinkBadass_Giorgio_C':'tink','BPChar_TinkBadassArmored_C':'tink','BPChar_TinkBadassArmored_Looter_C':'tink','BPChar_TinkBasic_C':'tink','BPChar_TinkBasic_Looter_C':'tink',
        'BPChar_TinkGoonBuddy_C':'tink','BPChar_TinkMotorcadeBigD_C':'tink','BPChar_TinkPsycho_C':'tink','BPChar_TinkPsycho_Looter_C':'tink','BPChar_TinkShotgun_C':'tink',
        'BPChar_TinkStagehand_C':'tink','BPChar_TinkSuicide_C':'tink','BPChar_TinkSuicide_Looter_C':'tink','BPChar_Trooper_Rare01c_C':'trooper','BPChar_Trooper_Rare01e_C':'trooper',
        'BPChar_TrooperBadass_C':'trooper','BPChar_TrooperBadassDark_C':'trooper','BPChar_TrooperBadassDarkRaid1_C':'trooper','BPChar_TrooperBasic_C':'trooper','BPChar_TrooperBasic_GigamindAdds_C':'trooper',
        'BPChar_TrooperBasicDark_C':'trooper','BPChar_TrooperBasicDarkRaid1_C':'trooper','BPChar_TrooperChumpMeleeDark_C':'trooper','BPChar_TrooperFlash_C':'trooper',
        'BPChar_TrooperFlashDark_C':'trooper','BPChar_TrooperFlashDarkRaid1_C':'trooper','BPChar_TrooperJetpack_C':'trooper','BPChar_TrooperMedicDark_C':'trooper','BPChar_TrooperMedicDarkRaid1_C':'trooper',
        'BPChar_TrooperMelee_C':'trooper','BPChar_TrooperMeleeDark_C':'trooper','BPChar_TrooperReflectDark_C':'trooper','BPChar_TrooperShotgun_C':'trooper','BPChar_TrooperShotgunDark_C':'trooper',
        'BPChar_TrooperStealthDark_C':'trooper','BPChar_VarkidAdult_C':'varki','BPChar_VarkidBadass_C':'varki','BPChar_VarkidHunt02_AdultB_C':'varki','BPChar_VarkidHunt02_LarvaB_C':'varki','BPChar_VarkidLarva_C':'varki',
        'BPChar_WeeLoaderBasic_C':'weeloader','BPSeatPawn_GunnersNest_C':'gunnersnest','DamageSource':'dmgsource','DamageSource_Artifact_C':'dmgartifact','DamageSource_BeastmasterPet_C':'beastmaster',
        'DamageSource_Bullet_AssaultRifle_BOTD_C':'assaultrifle','DamageSource_Bullet_AssaultRifle_C':'assaultrifle','DamageSource_Bullet_AssaultRifle_Kaos_C':'assaultrifle','DamageSource_Bullet_C':'bullet',
        'DamageSource_Bullet_Cloudkill_C':'cloudkill','DamageSource_Bullet_Heavy_C':'heavy','DamageSource_Bullet_Pistol_C':'pistol','DamageSource_Bullet_Shotgun_C':'shotgun',
        'DamageSource_Bullet_SMG_C':'smg','DamageSource_Bullet_SMG_KybsWorth_C':'smg','DamageSource_Bullet_Sniper_C':'sniper','DamageSource_CryoFallingImpact_C':'cryofall',
        'DamageSource_ElemTank_C':'elemtank','DamageSource_Environmental_Barrel_C':'environ','DamageSource_Environmental_C':'environ','DamageSource_Environmental_Electrical_C':'environ',
        'DamageSource_ForceOnly_IgnoreCooldown_C':'forceonly','DamageSource_GrenadeDoT_Chupa_C':'grenadedot','DamageSource_GroundSlam_C':'groundslam','DamageSource_IronBear_OffhandMelee_C':'gunner','DamageSource_Melee_C':'melee',
        'DamageSource_Melee_Overrun_C':'melee','DamageSource_Melee_Resonator_C':'melee','DamageSource_Projectile_C':'projectile','DamageSource_Skill_Beastmaster_HulkedOut_C':'beastmaster','DamageSource_Skill_Beastmaster_RakkAttack_C':'beastmaster',
        'DamageSource_Skill_Beastmaster_RakkAttack_CM_Raid1_C':'beastmaster','DamageSource_Skill_CalledShot_C':'calledshot','DamageSource_Skill_C':'skill','DamageSource_Skill_Gunner_FireSkagDen_C':'fireskagdengunner',
        'DamageSource_Skill_IronBear_GrenadeLauncher_C':'gunner','DamageSource_Skill_IronBear_Minigun_C':'gunner','DamageSource_Skill_IronBear_Railgun_C':'gunner','DamageSource_Skill_IronBear_VanquisherPod_C':'gunner',
        'DamageSource_Skill_Operative_PanicButton_C':'operative','DamageSource_Skill_Phasetrance_C':'siren','DamageSource_Skill_Phasetrance_Cast_C':'siren',
        'DamageSource_Skill_Phasetrance_Link_C':'siren','DamageSource_Slide_C':'slide','DamageSource_StuckGrenade_C':'stuckgrenade','DamageSource_Vehicle_C':'vehicle','DamageSource_Vehicle_HeavyWeapon_C':'vehicle',
        'DamageSource_Vehicle_Weapon_C':'vehicle','DamageSource_VehicleRamming_C':'vehicle','Vehicle_GunnerSeat_Technical_C':'vehicle','Vehicle_Outrunner_C':'vehicle','Vehicle_Outrunner_HoverWheels_C':'vehicle','Vehicle_Revolver_C':'vehicle',
        'Vehicle_Revolver_DualWheel_C':'vehicle','BPChar_OversphereDefenderDarkRaid1_C':'oversphere','BPChar_Heavy_BasicDark_C':'heavy','BPChar_TedioreTurret_PS_C':'tedturret'}

In [145]:
finale_js = finale_js.replace({'type':typed})

In [147]:
finale_js
finale_js.to_csv('wnajson.csv', index = False)

In [148]:
finale_js = finale_js.dropna()
finale_js.to_csv('wojson.csv', index = False)

# ANALYTICS.CSV

In [9]:
csv_path = 'C:/Users/sidew/Desktop/gearbox/analytics.csv'
df_strange = pd.read_csv(csv_path, error_bad_lines = False, sep = ',')

b'Skipping line 42: expected 10 fields, saw 16\nSkipping line 61: expected 10 fields, saw 16\nSkipping line 68: expected 10 fields, saw 16\nSkipping line 125: expected 10 fields, saw 16\nSkipping line 166: expected 10 fields, saw 16\nSkipping line 254: expected 10 fields, saw 16\nSkipping line 318: expected 10 fields, saw 11\nSkipping line 357: expected 10 fields, saw 11\nSkipping line 364: expected 10 fields, saw 16\nSkipping line 384: expected 10 fields, saw 17\nSkipping line 400: expected 10 fields, saw 16\nSkipping line 448: expected 10 fields, saw 16\nSkipping line 479: expected 10 fields, saw 16\nSkipping line 485: expected 10 fields, saw 16\nSkipping line 511: expected 10 fields, saw 11\nSkipping line 547: expected 10 fields, saw 11\nSkipping line 548: expected 10 fields, saw 16\nSkipping line 591: expected 10 fields, saw 11\nSkipping line 636: expected 10 fields, saw 11\nSkipping line 698: expected 10 fields, saw 11\nSkipping line 739: expected 10 fields, saw 11\nSkipping line 

b'Skipping line 133033: expected 10 fields, saw 16\nSkipping line 133212: expected 10 fields, saw 16\nSkipping line 133289: expected 10 fields, saw 16\nSkipping line 133326: expected 10 fields, saw 16\nSkipping line 133578: expected 10 fields, saw 16\nSkipping line 133646: expected 10 fields, saw 16\nSkipping line 133723: expected 10 fields, saw 11\nSkipping line 133899: expected 10 fields, saw 16\nSkipping line 133953: expected 10 fields, saw 11\nSkipping line 133977: expected 10 fields, saw 16\nSkipping line 134050: expected 10 fields, saw 15\nSkipping line 134119: expected 10 fields, saw 16\nSkipping line 134146: expected 10 fields, saw 16\nSkipping line 134181: expected 10 fields, saw 11\nSkipping line 134217: expected 10 fields, saw 16\nSkipping line 134309: expected 10 fields, saw 16\nSkipping line 134343: expected 10 fields, saw 11\nSkipping line 134346: expected 10 fields, saw 16\nSkipping line 134348: expected 10 fields, saw 16\nSkipping line 134360: expected 10 fields, saw 16

b'Skipping line 266034: expected 10 fields, saw 16\nSkipping line 266150: expected 10 fields, saw 16\nSkipping line 266203: expected 10 fields, saw 16\nSkipping line 266298: expected 10 fields, saw 16\nSkipping line 266317: expected 10 fields, saw 16\nSkipping line 266407: expected 10 fields, saw 16\nSkipping line 266423: expected 10 fields, saw 16\nSkipping line 266478: expected 10 fields, saw 16\nSkipping line 266523: expected 10 fields, saw 16\nSkipping line 266530: expected 10 fields, saw 16\nSkipping line 266637: expected 10 fields, saw 16\nSkipping line 266648: expected 10 fields, saw 16\nSkipping line 266710: expected 10 fields, saw 11\nSkipping line 266727: expected 10 fields, saw 16\nSkipping line 266783: expected 10 fields, saw 16\nSkipping line 266815: expected 10 fields, saw 11\nSkipping line 266956: expected 10 fields, saw 16\nSkipping line 267000: expected 10 fields, saw 16\nSkipping line 267043: expected 10 fields, saw 16\nSkipping line 267072: expected 10 fields, saw 16

b'Skipping line 399032: expected 10 fields, saw 16\nSkipping line 399133: expected 10 fields, saw 17\nSkipping line 399193: expected 10 fields, saw 16\nSkipping line 399270: expected 10 fields, saw 16\nSkipping line 399486: expected 10 fields, saw 11\nSkipping line 399681: expected 10 fields, saw 16\nSkipping line 399947: expected 10 fields, saw 16\nSkipping line 400024: expected 10 fields, saw 16\nSkipping line 400044: expected 10 fields, saw 11\nSkipping line 400134: expected 10 fields, saw 16\nSkipping line 400144: expected 10 fields, saw 16\nSkipping line 400187: expected 10 fields, saw 16\nSkipping line 400374: expected 10 fields, saw 11\nSkipping line 400463: expected 10 fields, saw 11\nSkipping line 400491: expected 10 fields, saw 16\nSkipping line 400617: expected 10 fields, saw 11\nSkipping line 400639: expected 10 fields, saw 16\nSkipping line 400668: expected 10 fields, saw 16\nSkipping line 400748: expected 10 fields, saw 16\nSkipping line 400800: expected 10 fields, saw 16

C:\Users\sidew\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Skipping line 499485: expected 10 fields, saw 22

## Skipping line 500861: expected 10 fields, saw 22

In [10]:
with open(csv_path) as f:
    len_csv = sum(1 for line in f)

nu = len_csv - len(df_strange)
print(len_csv)
print(len(df_strange))
print(nu)

1765540
1241349
524191


## Specify 22 columns while reading data

In [11]:
column_names = [i for i in range(22)]

In [12]:
df_strange2 = pd.read_csv(csv_path, names=column_names, sep = ',', skiprows=1, dtype ='unicode')

In [13]:
with open(csv_path) as f:
    len_csv = sum(1 for line in f)

nu2 = len_csv - len(df_strange2)
print(len_csv)
print(len(df_strange2))
print(nu2)

1765540
1248887
516653


In [14]:
df_svg = df_strange2

In [15]:
df_svg = df_svg.fillna('*')

## MULTIPLE OBSERVATIONS PRESENT IN A SINGLE ROW, SUPPOSED TO BE SEPARATED BY A '\n' CHARACTER, DATA FROM ADJACENT COLUMNS LEAKING INTO THE NEXT COLUMNS - UNCLEAN DATA

In [16]:
df_svg

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,2023,11,20 18:00:00,"000019b435911a2c74c499fcbc739a01,3,BPChar_Beas...",3,BPChar_Operative_C,Monastery_P,LevelUp,17198,*,...,*,*,*,*,*,*,*,*,*,*
1,2023,10,11 19:00:00,"00003c6bb1dd994663e3b1a92a04d8a0,3,BPChar_Gunn...",3,BPChar_Siren_C,Loader,LevelUp,75408,*,...,*,*,*,*,*,*,*,*,*,*
2,2023,10,18 19:00:00,"000033b91707994ae2eab63ed7bf5f24,2,BPChar_Sire...",2,BPChar_Operative_C,Monastery_P,LevelUp,201201,*,...,*,*,*,*,*,*,*,*,*,*
3,2023,11,17 18:00:00,"0000431a3f19bb896d3d82a33c7a208f,2,BPChar_Gunn...",2,BPChar_Operative_C,Prologue_P,LevelUp,12370,*,...,*,*,*,*,*,*,*,*,*,*
4,2023,11,13 18:00:00,"0000466dd39896eae8fc0e068175696f,1,BPChar_Oper...",1,BPChar_Beastmaster_C,COVSlaughter_P,LevelUp,90574,*,...,*,*,*,*,*,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248882,2024,01,22 18:00:00,00002334b514baf2f1770078786a26a8,1,BPChar_Beastmaster_C,Wetlands_P,LevelUp,68096,27,...,*,*,*,*,*,*,*,*,*,*
1248883,2024,01,22 18:00:00,00001b8c5ed90905190d9d355fc09c60,1,BPChar_IronBear_DakkaBear_C,Motorcade_P,LevelUp,108595,37,...,*,*,*,*,*,*,*,*,*,*
1248884,2024,01,22 18:00:00,0000436f5576dd5a2507a1c48e43e1d0,1,BPChar_Siren_C,Sanctuary3_P,LevelUp,49557,18,...,*,*,*,*,*,*,*,*,*,*
1248885,2024,01,23 18:00:00,000041c68dfa01a5045c64ebc339e4f5,1,BPChar_Beastmaster_C,Towers_P,LevelUp,92478,*,...,*,*,*,*,*,*,*,*,*,*


## CONVERTING ALL COLUMNS TO STRING TYPE AND JOINING ALL OF THEM INTO A SINGLE COLUMN USING ',' TO DISTINGUISH BETWEEN ENTRIES OF MULTIPLE COLUMNS

In [17]:
hj = list(df_svg.columns)
for k in hj:
    df_svg[k] = df_svg[k].astype(str)

In [18]:
df_svg['NC'] = df_svg.iloc[:,:].apply(lambda x: ','.join(x), axis=1)

## SPLIT EACH ENTRY PRESENT IN THIS NEW COLUMN AT THE '\n' LINE BREAK TO SEPARATE INDIVIDUAL OBSERVATIONS

In [19]:
df_svg['NC'][0].split('\n')

['2023,11,20 18:00:00,000019b435911a2c74c499fcbc739a01,3,BPChar_Beastmaster_C,Prologue_P,LevelUp,7866,',
 '2023,11,22 18:00:00,000002d0c1df3e77128921da89a5d655,3,BPChar_Operative_C,Monastery_P,LevelUp,17198,*,*,*,*,*,*,*,*,*,*,*,*,*']

## Each row must contain 10 columns -                                                                                               0:Year, 1:Month, 2:DateTime, 3:PlayerID, 4:Platform, 5:Character, 6:Map, 7:Event, 8:PlayedTime, 9:Level

In [20]:
lis = []
xtra = []
bad_ind = []
for i,el in enumerate(df_svg['NC']):
    l = el.split("\n")
    for x in l:
        y = x.split(',')
        if len(y) > 10:         # SOME ROWS HAVE MORE THAN 10 COLUMNS - PUT INTO A SEPARATE DF CALLED 'xtra_df' TO EXAMINE
            xtra.append(y)
            bad_ind.append(i)
        else:
            lis.append(y)        # ROWS WITH EXACTLY 10 COLUMNS - THAT FIT OUR ASSUMPTION - PUT INTO A DF CALLED 'n_df'

## CREATE NEW DATAFRAME WITH THE REQUIRED NUMBER OF COLUMNS AND WITH EACH COLUMN CONTAINING ONLY THE APPROPRIATE ENTRY FOR THAT COLUMN

In [21]:
n_df = pd.DataFrame(lis)
n_df

,0,1,2,3,4,5,6,7,8,9
0,2023,11,20 18:00:00,000019b435911a2c74c499fcbc739a01,3,BPChar_Beastmaster_C,Prologue_P,LevelUp,7866,
1,2023,10,11 19:00:00,00003c6bb1dd994663e3b1a92a04d8a0,3,BPChar_Gunner_C,AtlasHQ_P,LevelUp,28691,
2,2023,10,18 19:00:00,000033b91707994ae2eab63ed7bf5f24,2,BPChar_Siren_C,ProvingGrounds_Trial4_P,LevelUp,178546,
3,2023,11,17 18:00:00,0000431a3f19bb896d3d82a33c7a208f,2,BPChar_Gunner_C,COVSlaughter_P,LevelUp,61067,
4,2023,11,13 18:00:00,0000466dd39896eae8fc0e068175696f,1,BPChar_Operative_C,COVSlaughter_P,LevelUp,400079,
...,...,...,...,...,...,...,...,...,...,...
516800,2023,11,20 18:00:00,00002a7829c143184a455477804ac143,3,BPChar_Beastmaster_C,Wetlands_P,LevelUp,43509,
516801,2023,10,29 19:00:00,00000f66dec0e799fe8e10b5e00b672f,3,BPChar_Siren_C,Mine_P,LevelUp,134518,
516802,2023,10,10 19:00:00,000018a507f922d9b484239d48de7a8b,2,BPChar_Beastmaster_C,Sanctuary3_P,LevelUp,81907,
516803,2023,10,29 19:00:00,00002f5845c7ba6351305a74e1a0bf1e,2,BPChar_Operative_C,Loader,LevelUp,7523,


In [22]:
n_df.isnull().sum()

0    0
1    0
2    0
3    0
4    2
5    2
6    2
7    2
8    2
9    5
dtype: int64

In [24]:
xtra_df = pd.DataFrame(xtra)
xtra_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2023,11,22 18:00:00,000002d0c1df3e77128921da89a5d655,3,BPChar_Operative_C,Monastery_P,LevelUp,17198,*,...,*,*,*,*,None,None,None,None,None,None
1,2023,11,21 18:00:00,00002b8968bac2feb81eabf0a509642b,3,BPChar_Siren_C,Loader,LevelUp,75408,*,...,*,*,*,*,None,None,None,None,None,None
2,2023,10,12 19:00:00,0000234277ff548a5e009ab7467121b6,2,BPChar_Operative_C,Monastery_P,LevelUp,201201,*,...,*,*,*,*,None,None,None,None,None,None
3,2023,10,12 19:00:00,0000363f80d92a54ca6ceddffdad1596,2,BPChar_Operative_C,Prologue_P,LevelUp,12370,*,...,*,*,*,*,None,None,None,None,None,None
4,2023,11,14 18:00:00,000008c14c4583855dafe14734370de8,1,BPChar_Beastmaster_C,COVSlaughter_P,LevelUp,90574,*,...,*,*,*,*,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1248728,2024,01,22 18:00:00,00002334b514baf2f1770078786a26a8,1,BPChar_Beastmaster_C,Wetlands_P,LevelUp,68096,27,...,*,*,*,*,None,None,None,None,None,None
1248729,2024,01,22 18:00:00,00001b8c5ed90905190d9d355fc09c60,1,BPChar_IronBear_DakkaBear_C,Motorcade_P,LevelUp,108595,37,...,*,*,*,*,None,None,None,None,None,None
1248730,2024,01,22 18:00:00,0000436f5576dd5a2507a1c48e43e1d0,1,BPChar_Siren_C,Sanctuary3_P,LevelUp,49557,18,...,*,*,*,*,None,None,None,None,None,None
1248731,2024,01,23 18:00:00,000041c68dfa01a5045c64ebc339e4f5,1,BPChar_Beastmaster_C,Towers_P,LevelUp,92478,*,...,*,*,*,*,None,None,None,None,None,None


In [24]:
xtra_df[27].unique()

array([None, '*'], dtype=object)

### None of the extra columns have any information contained within them. Safe to drop.

In [25]:
res_df = xtra_df.iloc[:,0:10]
res_df

,0,1,2,3,4,5,6,7,8,9
0,2023,11,22 18:00:00,000002d0c1df3e77128921da89a5d655,3,BPChar_Operative_C,Monastery_P,LevelUp,17198,*
1,2023,11,21 18:00:00,00002b8968bac2feb81eabf0a509642b,3,BPChar_Siren_C,Loader,LevelUp,75408,*
2,2023,10,12 19:00:00,0000234277ff548a5e009ab7467121b6,2,BPChar_Operative_C,Monastery_P,LevelUp,201201,*
3,2023,10,12 19:00:00,0000363f80d92a54ca6ceddffdad1596,2,BPChar_Operative_C,Prologue_P,LevelUp,12370,*
4,2023,11,14 18:00:00,000008c14c4583855dafe14734370de8,1,BPChar_Beastmaster_C,COVSlaughter_P,LevelUp,90574,*
...,...,...,...,...,...,...,...,...,...,...
1248728,2024,01,22 18:00:00,00002334b514baf2f1770078786a26a8,1,BPChar_Beastmaster_C,Wetlands_P,LevelUp,68096,27
1248729,2024,01,22 18:00:00,00001b8c5ed90905190d9d355fc09c60,1,BPChar_IronBear_DakkaBear_C,Motorcade_P,LevelUp,108595,37
1248730,2024,01,22 18:00:00,0000436f5576dd5a2507a1c48e43e1d0,1,BPChar_Siren_C,Sanctuary3_P,LevelUp,49557,18
1248731,2024,01,23 18:00:00,000041c68dfa01a5045c64ebc339e4f5,1,BPChar_Beastmaster_C,Towers_P,LevelUp,92478,*


In [26]:
count = 0
for o in res_df[8]:
    m = o.lower()
    if m.islower():
        count += 1        # CHECKING IF ANY ALPHABETS ARE PRESENT IN THE PLAYEDTIME COLUMN, INDICATING AN ERROR IN THE PROCESS
count

0

In [27]:
res_df.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

In [28]:
fina_df = n_df.append(res_df)
fina_df = fina_df.reset_index()
del fina_df['index']

## ASSIGN THE APPROPRIATE COLUMN NAMES TO OBTAIN RELATIVELY CLEANER DATASET

In [29]:
fina_df.columns = ['YEAR','MONTH','DATE','PLAYERID','PLATFORM','CHARACTERID','MAP','EVENT','PLAYEDTIME','LEVEL']
fina_df

,YEAR,MONTH,DATE,PLAYERID,PLATFORM,CHARACTERID,MAP,EVENT,PLAYEDTIME,LEVEL
0,2023,11,20 18:00:00,000019b435911a2c74c499fcbc739a01,3,BPChar_Beastmaster_C,Prologue_P,LevelUp,7866,
1,2023,10,11 19:00:00,00003c6bb1dd994663e3b1a92a04d8a0,3,BPChar_Gunner_C,AtlasHQ_P,LevelUp,28691,
2,2023,10,18 19:00:00,000033b91707994ae2eab63ed7bf5f24,2,BPChar_Siren_C,ProvingGrounds_Trial4_P,LevelUp,178546,
3,2023,11,17 18:00:00,0000431a3f19bb896d3d82a33c7a208f,2,BPChar_Gunner_C,COVSlaughter_P,LevelUp,61067,
4,2023,11,13 18:00:00,0000466dd39896eae8fc0e068175696f,1,BPChar_Operative_C,COVSlaughter_P,LevelUp,400079,
...,...,...,...,...,...,...,...,...,...,...
1765533,2024,01,22 18:00:00,00002334b514baf2f1770078786a26a8,1,BPChar_Beastmaster_C,Wetlands_P,LevelUp,68096,27
1765534,2024,01,22 18:00:00,00001b8c5ed90905190d9d355fc09c60,1,BPChar_IronBear_DakkaBear_C,Motorcade_P,LevelUp,108595,37
1765535,2024,01,22 18:00:00,0000436f5576dd5a2507a1c48e43e1d0,1,BPChar_Siren_C,Sanctuary3_P,LevelUp,49557,18
1765536,2024,01,23 18:00:00,000041c68dfa01a5045c64ebc339e4f5,1,BPChar_Beastmaster_C,Towers_P,LevelUp,92478,*


In [30]:
nulls_in_fina = fina_df[fina_df.isnull().any(axis=1)]
nulls_in_fina

,YEAR,MONTH,DATE,PLAYERID,PLATFORM,CHARACTERID,MAP,EVENT,PLAYEDTIME,LEVEL
8319,2024,01,15 18:00:00,0000416be1f755db2dacc9dba8e3be82,3,BPChar_Beastmaster_C,Trashtown_P,LevelUp,65363,None
141241,2023,10,17 19:00:00,000002bd307cf2e96d0bf772f3c62923,1,BPChar_Beastmaster_C,FinalBoss_P,LevelUp,431951,None
188051,2023,10,30 19:00:00,00001f7a6b96dfbfa0244e7cf2075c88,None,None,None,None,None,None
322353,2023,11,28 18:00:00,000021dbe9d2420268a7e0f9e6e33cd9,1,NONE,COVSlaughter_P,LevelUp,4000,None
498934,2023,11,01 19:00:00,000021b97d8247f0bda41dbc19d7510d,None,None,None,None,None,None


In [31]:
fina_df.isna().sum()

YEAR           0
MONTH          0
DATE           0
PLAYERID       0
PLATFORM       2
CHARACTERID    2
MAP            2
EVENT          2
PLAYEDTIME     2
LEVEL          5
dtype: int64

# GOING THROUGH INDIVIDUAL COLUMNS TO VERIFY INTEGRITY

### LEVEL

In [36]:
fina_df['LEVEL'].unique()

array([        nan, 3.30000e+01, 5.00000e+01, 1.50000e+01, 1.40000e+01,
       1.70000e+01, 4.10000e+01, 7.00000e+00, 2.00000e+00, 3.20000e+01,
       3.90000e+01, 6.00000e+00, 1.20000e+01, 4.80000e+01, 2.40000e+01,
       1.10000e+01, 3.00000e+00, 2.20000e+01, 1.80000e+01, 1.30000e+01,
       1.90000e+01, 9.00000e+00, 8.00000e+00, 3.10000e+01, 4.40000e+01,
       4.30000e+01, 3.00000e+01, 2.80000e+01, 2.00000e+01, 2.90000e+01,
       1.00000e+00, 1.00000e+01, 4.70000e+01, 1.60000e+01, 4.00000e+00,
       5.00000e+00, 2.60000e+01, 3.50000e+01, 3.70000e+01, 2.10000e+01,
       2.70000e+01, 4.50000e+01, 3.40000e+01, 3.60000e+01, 4.60000e+01,
       4.00000e+01, 2.30000e+01, 3.80000e+01, 4.90000e+01, 4.20000e+01,
       2.50000e+01, 1.14404e+09])

In [33]:
fina_df['LEVEL'].fillna('-1', inplace = True)
fina_df['LEVEL'].replace({'':'-1','*':'-1'}, inplace = True)

In [34]:
fina_df['LEVEL'] = pd.to_numeric(fina_df['LEVEL'])

In [35]:
fina_df['LEVEL'].replace(-1, np.nan, inplace = True)
fina_df['LEVEL'].isnull().sum()

1567676

### PLAYEDTIME

In [37]:
fina_df['PLAYEDTIME'] = pd.to_numeric(fina_df['PLAYEDTIME'])
fina_df['PLAYEDTIME'].isnull().sum()

2

In [38]:
fina_df['PLAYEDTIME'].unique()

array([   7866.,   28691.,  178546., ...,  848000.,  867430., 1177340.])

### EVENT

In [39]:
fina_df['EVENT'].fillna(np.nan, inplace = True)
fina_df['EVENT'].unique()

array(['LevelUp', nan], dtype=object)

### MAP

In [40]:
fina_df['MAP'].fillna(np.nan, inplace = True)
fina_df['MAP'].unique()

array(['Prologue_P', 'AtlasHQ_P', 'ProvingGrounds_Trial4_P',
       'COVSlaughter_P', 'CityVault_P', 'Loader', 'Desolate_P', 'Crypt_P',
       'Wetlands_P', 'Recruitment_P', 'MotorcadeInterior_P',
       'MarshFields_P', 'TechSlaughter_P', 'WetlandsBoss_P', 'Desert_P',
       'Mine_P', 'Convoy_P', 'CreatureSlaughter_P',
       'ProvingGrounds_Trial5_P', 'Monastery_P', 'Outskirts_P', 'Raid_P',
       'Towers_P', 'OrbitalPlatform_P', 'Watership_P', 'Prison_P',
       'Motorcade_P', 'FinalBoss_P', 'Sanctuary3_P',
       'ProvingGrounds_Trial8_P', 'City_P', 'Sacrifice_P', 'Mansion_P',
       'MotorcadeFestival_P', 'Desertvault_P', 'ProvingGrounds_Trial6_P',
       'CityBoss_P', 'ProvingGrounds_Trial1_P', 'WetlandsVault_P',
       'BloodyHarvest_P', 'DesertBoss_P', 'Beach_P',
       'ProvingGrounds_Trial7_P', 'Strip_P', 'Trashtown_P', 'Core_P',
       'CasinoIntro_P', 'TowerLair_P', 'Impound_P', 'MenuMap_P', nan],
      dtype=object)

### CHARACTERID

In [41]:
# len(fina_df[fina_df['CHARACTERID']=='*']) #1
fina_df['CHARACTERID'].replace(['NONE', '*'], np.nan, inplace = True)
fina_df['CHARACTERID'].fillna(np.nan, inplace = True)
fina_df['CHARACTERID'].unique()

array(['BPChar_Beastmaster_C', 'BPChar_Gunner_C', 'BPChar_Siren_C',
       'BPChar_Operative_C', 'Vehicle_Outrunner_HoverWheels_C',
       'Vehicle_Technical_BigWheels_C', 'Vehicle_Technical_C', nan,
       'BPChar_IronBear_C', 'Vehicle_Outrunner_C',
       'BPChar_IronBear_DakkaBear_C', 'Vehicle_Revolver_C',
       'Vehicle_Revolver_DualWheel_C', 'Vehicle_Outrunner_ZipWheels_C',
       'Vehicle_Revolver_HoverWheels_C',
       'Vehicle_Outrunner_TwitchyWheels_C', 'IO_GroundTurret_C',
       'Vehicle_GunnerSeat_Outrunner_C',
       'Vehicle_Technical_HoverWheels_C',
       'Vehicle_Technical_BarbedWheels_C',
       'Vehicle_GunnerSeat_Technical_C', 'Vehicle_Revolver_WideWheel_C',
       'BPSeatPawn_GunnersNest_C'], dtype=object)

### PLATFORM

In [42]:
fina_df['PLATFORM'] = pd.to_numeric(fina_df['PLATFORM'])
fina_df['PLATFORM'].unique()

array([ 3.,  2.,  1.,  4.,  5., nan])

### PLAYERID

In [43]:
fina_df['PLAYERID'].isnull().sum()
len(fina_df[fina_df['PLAYERID'] == 'None'])

0

### TIMESTAMP

In [44]:
fina_df['TIMESTAMP'] = fina_df['YEAR']+'-'+fina_df['MONTH']+'-'+fina_df['DATE']

In [45]:
fina_df['TIMESTAMP'][0]
fina_df['TIMESTAMP'] = [datetime.strptime(u, '%Y-%m-%d %H:%M:%S') for u in fina_df['TIMESTAMP']]

## REARRANGING COLUMNS INTO A MORE MEANINGFUL ORDER

In [173]:
final_df = fina_df.iloc[:,[10,3,4,5,6,7,8,9]]

In [77]:
# pd.options.display.float_format = '{:,.0f}'.format
final_df

,TIMESTAMP,PLAYERID,PLATFORM,CHARACTERID,MAP,EVENT,PLAYEDTIME,LEVEL
0,2023-11-20 18:00:00,000019b435911a2c74c499fcbc739a01,3.0,BPChar_Beastmaster_C,Prologue_P,LevelUp,7866.0,NaN
1,2023-10-11 19:00:00,00003c6bb1dd994663e3b1a92a04d8a0,3.0,BPChar_Gunner_C,AtlasHQ_P,LevelUp,28691.0,NaN
2,2023-10-18 19:00:00,000033b91707994ae2eab63ed7bf5f24,2.0,BPChar_Siren_C,ProvingGrounds_Trial4_P,LevelUp,178546.0,NaN
3,2023-11-17 18:00:00,0000431a3f19bb896d3d82a33c7a208f,2.0,BPChar_Gunner_C,COVSlaughter_P,LevelUp,61067.0,NaN
4,2023-11-13 18:00:00,0000466dd39896eae8fc0e068175696f,1.0,BPChar_Operative_C,COVSlaughter_P,LevelUp,400079.0,NaN
...,...,...,...,...,...,...,...,...
1765533,2024-01-22 18:00:00,00002334b514baf2f1770078786a26a8,1.0,BPChar_Beastmaster_C,Wetlands_P,LevelUp,68096.0,27.0
1765534,2024-01-22 18:00:00,00001b8c5ed90905190d9d355fc09c60,1.0,BPChar_IronBear_DakkaBear_C,Motorcade_P,LevelUp,108595.0,37.0
1765535,2024-01-22 18:00:00,0000436f5576dd5a2507a1c48e43e1d0,1.0,BPChar_Siren_C,Sanctuary3_P,LevelUp,49557.0,18.0
1765536,2024-01-23 18:00:00,000041c68dfa01a5045c64ebc339e4f5,1.0,BPChar_Beastmaster_C,Towers_P,LevelUp,92478.0,NaN


In [193]:
final_df.isna().sum()

TIMESTAMP            0
PLAYERID             0
PLATFORM             2
CHARACTERID      98015
MAP                  2
EVENT                2
PLAYEDTIME           2
LEVEL          1567676
dtype: int64

## COMBINING FACTORS IN A COLUMN TO REDUCE THE TOTAL NUMBER OF FACTORS IN EACH COLUMN - DONE IN A MEANINGFUL MANNER W.R.T THE DOMAIN

In [177]:
final_df['CHARACTERID'].unique()

array(['beastmaster', 'gunner', 'siren', 'operative', 'outrunner',
       'technical', nan, 'revolver', 'groundturret', 'gunnersnest'],
      dtype=object)

In [162]:
charid = {'BPChar_Beastmaster_C':'beastmaster', 'BPChar_Gunner_C':'gunner','BPChar_IronBear_C':'gunner','BPChar_IronBear_DakkaBear_C':'gunner',
          'BPChar_Siren_C':'siren', 'BPChar_Operative_C':'operative',
          'Vehicle_Outrunner_HoverWheels_C':'outrunner','Vehicle_Outrunner_C':'outrunner','Vehicle_Outrunner_ZipWheels_C':'outrunner','Vehicle_Outrunner_TwitchyWheels_C':'outrunner','Vehicle_GunnerSeat_Outrunner_C':'outrunner',
          'Vehicle_Technical_BigWheels_C':'technical','Vehicle_Technical_C':'technical','Vehicle_Technical_HoverWheels_C':'technical','Vehicle_Technical_BarbedWheels_C':'technical','Vehicle_GunnerSeat_Technical_C':'technical',
          'Vehicle_Revolver_C':'revolver','Vehicle_Revolver_DualWheel_C':'revolver','Vehicle_Revolver_HoverWheels_C':'revolver','Vehicle_Revolver_WideWheel_C':'revolver',
          'IO_GroundTurret_C':'groundturret','BPSeatPawn_GunnersNest_C':'gunnersnest'}

In [174]:
final_df = final_df.replace({'CHARACTERID' : charid})

In [176]:
final_df['MAP'].unique()

array(['Prologue_P', 'AtlasHQ_P', 'provinggrounds', 'circleofslaughter',
       'city', 'Loader', 'Desolate_P', 'Crypt_P', 'wetlands',
       'Recruitment_P', 'motorcade', 'MarshFields_P', 'desert', 'Mine_P',
       'Convoy_P', 'Monastery_P', 'Outskirts_P', 'Raid_P', 'Towers_P',
       'OrbitalPlatform_P', 'Watership_P', 'Prison_P', 'FinalBoss_P',
       'Sanctuary3_P', 'Sacrifice_P', 'Mansion_P', 'BloodyHarvest_P',
       'Beach_P', 'Strip_P', 'Trashtown_P', 'Core_P', 'CasinoIntro_P',
       'TowerLair_P', 'Impound_P', 'MenuMap_P', nan], dtype=object)

In [165]:
maps = {'ProvingGrounds_Trial4_P':'provinggrounds','ProvingGrounds_Trial5_P':'provinggrounds','ProvingGrounds_Trial8_P':'provinggrounds',
       'ProvingGrounds_Trial6_P':'provinggrounds','ProvingGrounds_Trial1_P':'provinggrounds','ProvingGrounds_Trial7_P':'provinggrounds',
       'COVSlaughter_P':'circleofslaughter','CreatureSlaughter_P':'circleofslaughter','TechSlaughter_P':'circleofslaughter','MotorcadeInterior_P':'motorcade',
       'Motorcade_P':'motorcade','MotorcadeFestival_P':'motorcade','City_P':'city','CityVault_P':'city','CityBoss_P':'city','Desert_P':'desert',
       'Desertvault_P':'desert','DesertBoss_P':'desert','WetlandsBoss_P':'wetlands','Wetlands_P':'wetlands','WetlandsVault_P':'wetlands'}

In [175]:
final_df = final_df.replace({'MAP' : maps})

## MINOR MODIFICATIONS TO DATA TYPES AND DEALING WITH MISSING VALUES

In [187]:
final_df['PLATFORM']=final_df['PLATFORM'].astype(str)

In [189]:
final_df['PLATFORM'] = final_df['PLATFORM'].replace('nan',np.nan)

In [191]:
final_df['PLATFORM'].unique()

array(['3.0', '2.0', '1.0', '4.0', '5.0', nan], dtype=object)

## WRITING THE MODIFIED DATAFRAME INTO A CSV TO WORK UPON

In [192]:
final_df.to_csv('withna.csv', index = False)

In [194]:
full_df = final_df.iloc[:,0:7]
full_df = full_df.dropna()
full_df.isna().sum()

TIMESTAMP      0
PLAYERID       0
PLATFORM       0
CHARACTERID    0
MAP            0
EVENT          0
PLAYEDTIME     0
dtype: int64

In [195]:
full_df.to_csv('wona.csv',index = False)